# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 13 2023 9:41AM,264611,15,30017173,"Alliance Pharma, Inc.",Released
1,Jun 13 2023 9:40AM,264584,15,30017236,"Alliance Pharma, Inc.",Executing
2,Jun 13 2023 9:40AM,264584,15,30017234,"Alliance Pharma, Inc.",Executing
3,Jun 13 2023 9:40AM,264584,15,30017235,"Alliance Pharma, Inc.",Released
4,Jun 13 2023 9:40AM,264637,16,9568555,Sartorius Lab Products and Service,Released
5,Jun 13 2023 9:34AM,264559,15,30017229,"Alliance Pharma, Inc.",Executing
6,Jun 13 2023 9:34AM,264559,15,30017230,"Alliance Pharma, Inc.",Executing
7,Jun 13 2023 9:34AM,264559,15,30017231,"Alliance Pharma, Inc.",Executing
8,Jun 13 2023 9:34AM,264559,15,30017232,"Alliance Pharma, Inc.",Released
9,Jun 13 2023 9:34AM,264559,15,30017205,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
11,264623,Released,7
12,264625,Released,5
13,264627,Released,1
14,264628,Released,11
15,264637,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
264623,19.0,7.0
264625,NaN,5.0
264627,NaN,1.0
264628,NaN,11.0
264637,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
264559,17.0,11.0
264584,2.0,1.0
264588,1.0,0.0
264604,1.0,0.0
264611,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
264559,17,11
264584,2,1
264588,1,0
264604,1,0
264611,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,264559,17,11
1,264584,2,1
2,264588,1,0
3,264604,1,0
4,264611,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,264559,17,11
1,264584,2,1
2,264588,1,
3,264604,1,
4,264611,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 13 2023 9:41AM,264611,15,"Alliance Pharma, Inc."
1,Jun 13 2023 9:40AM,264584,15,"Alliance Pharma, Inc."
4,Jun 13 2023 9:40AM,264637,16,Sartorius Lab Products and Service
5,Jun 13 2023 9:34AM,264559,15,"Alliance Pharma, Inc."
33,Jun 13 2023 8:56AM,264628,10,Hush Hush
44,Jun 13 2023 8:55AM,264627,16,Sartorius Lab Products and Service
45,Jun 13 2023 8:46AM,264625,10,Emerginnova
50,Jun 13 2023 8:39AM,264623,20,"Exact-Rx, Inc."
76,Jun 13 2023 8:33AM,264622,22,FUJIFILM Diosynth Biotechnologies Texas LLC
77,Jun 13 2023 7:44AM,264620,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 13 2023 9:41AM,264611,15,"Alliance Pharma, Inc.",,1
1,Jun 13 2023 9:40AM,264584,15,"Alliance Pharma, Inc.",2,1
2,Jun 13 2023 9:40AM,264637,16,Sartorius Lab Products and Service,,1
3,Jun 13 2023 9:34AM,264559,15,"Alliance Pharma, Inc.",17,11
4,Jun 13 2023 8:56AM,264628,10,Hush Hush,,11
5,Jun 13 2023 8:55AM,264627,16,Sartorius Lab Products and Service,,1
6,Jun 13 2023 8:46AM,264625,10,Emerginnova,,5
7,Jun 13 2023 8:39AM,264623,20,"Exact-Rx, Inc.",19,7
8,Jun 13 2023 8:33AM,264622,22,FUJIFILM Diosynth Biotechnologies Texas LLC,1,
9,Jun 13 2023 7:44AM,264620,19,"AdvaGen Pharma, Ltd",10,8


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 13 2023 9:41AM,264611,15,"Alliance Pharma, Inc.",1,
1,Jun 13 2023 9:40AM,264584,15,"Alliance Pharma, Inc.",1,2
2,Jun 13 2023 9:40AM,264637,16,Sartorius Lab Products and Service,1,
3,Jun 13 2023 9:34AM,264559,15,"Alliance Pharma, Inc.",11,17
4,Jun 13 2023 8:56AM,264628,10,Hush Hush,11,
5,Jun 13 2023 8:55AM,264627,16,Sartorius Lab Products and Service,1,
6,Jun 13 2023 8:46AM,264625,10,Emerginnova,5,
7,Jun 13 2023 8:39AM,264623,20,"Exact-Rx, Inc.",7,19
8,Jun 13 2023 8:33AM,264622,22,FUJIFILM Diosynth Biotechnologies Texas LLC,,1
9,Jun 13 2023 7:44AM,264620,19,"AdvaGen Pharma, Ltd",8,10


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 13 2023 9:41AM,264611,15,"Alliance Pharma, Inc.",1,
1,Jun 13 2023 9:40AM,264584,15,"Alliance Pharma, Inc.",1,2
2,Jun 13 2023 9:40AM,264637,16,Sartorius Lab Products and Service,1,
3,Jun 13 2023 9:34AM,264559,15,"Alliance Pharma, Inc.",11,17
4,Jun 13 2023 8:56AM,264628,10,Hush Hush,11,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 13 2023 9:41AM,264611,15,"Alliance Pharma, Inc.",1.0,NaN
1,Jun 13 2023 9:40AM,264584,15,"Alliance Pharma, Inc.",1.0,2.0
2,Jun 13 2023 9:40AM,264637,16,Sartorius Lab Products and Service,1.0,NaN
3,Jun 13 2023 9:34AM,264559,15,"Alliance Pharma, Inc.",11.0,17.0
4,Jun 13 2023 8:56AM,264628,10,Hush Hush,11.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 13 2023 9:41AM,264611,15,"Alliance Pharma, Inc.",1.0,0.0
1,Jun 13 2023 9:40AM,264584,15,"Alliance Pharma, Inc.",1.0,2.0
2,Jun 13 2023 9:40AM,264637,16,Sartorius Lab Products and Service,1.0,0.0
3,Jun 13 2023 9:34AM,264559,15,"Alliance Pharma, Inc.",11.0,17.0
4,Jun 13 2023 8:56AM,264628,10,Hush Hush,11.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,529253,16.0,0.0
15,793754,13.0,19.0
16,793868,2.0,1.0
19,529208,8.0,11.0
20,264623,7.0,19.0
22,264622,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,529253,16.0,0.0
1,15,793754,13.0,19.0
2,16,793868,2.0,1.0
3,19,529208,8.0,11.0
4,20,264623,7.0,19.0
5,22,264622,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,16.0,0.0
1,15,13.0,19.0
2,16,2.0,1.0
3,19,8.0,11.0
4,20,7.0,19.0
5,22,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,16.0
1,15,Released,13.0
2,16,Released,2.0
3,19,Released,8.0
4,20,Released,7.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,22
Status,,,,,,
Executing,0.0,19.0,1.0,11.0,19.0,1.0
Released,16.0,13.0,2.0,8.0,7.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,22
0,Executing,0.0,19.0,1.0,11.0,19.0,1.0
1,Released,16.0,13.0,2.0,8.0,7.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,22
0,Executing,0.0,19.0,1.0,11.0,19.0,1.0
1,Released,16.0,13.0,2.0,8.0,7.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()